# Simulation des Scénarios Futurs - UBEM Résidentiel Québec

Ce notebook permet de simuler tous les scénarios futurs (2035 et 2050) pour le modèle UBEM Résidentiel Québec. Il offre la possibilité de lancer des simulations individuelles ou en batch pour tous les scénarios disponibles.

## Structure des scénarios

Les scénarios sont organisés selon la nomenclature suivante :
```
{année}_{type_climat}_{croissance_électrification}_{efficacité}
```

- **Année** : 2035 ou 2050
- **Type de climat** : warm (chaud), typical (normal), cold (froid)
- **Croissance et électrification** : PV (Projection Verte), UB (Ultra Bas carbone)
- **Efficacité** : E (standard), EE (maximale)

Par exemple : **2035_warm_PV_E** représente un scénario pour 2035 avec un climat chaud, une projection verte standard et une efficacité standard.

In [1]:
# Imports des modules nécessaires
import os
import sys
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from datetime import datetime
import subprocess
from IPython.display import display, HTML, clear_output

# Ajouter le chemin du projet pour importer les modules
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Imports des modules du projet
from src.config import config

## Configuration et Initialisation

In [2]:
# Afficher tous les scénarios disponibles
print("Scénarios disponibles :")
for scenario in config.future_scenarios.SCENARIOS:
    description = config.future_scenarios.SCENARIO_NAMES.get(scenario, scenario)
    print(f"- {scenario}: {description}")

Scénarios disponibles :
- 2035_warm_PV_E: 2035 - Année chaude - Projection verte - Efficacité standard
- 2035_warm_PV_EE: 2035 - Année chaude - Projection verte - Efficacité maximale
- 2035_warm_UB_E: 2035 - Année chaude - Ultra bas carbone - Efficacité standard
- 2035_warm_UB_EE: 2035 - Année chaude - Ultra bas carbone - Efficacité maximale
- 2035_typical_PV_E: 2035 - Année normale - Projection verte - Efficacité standard
- 2035_typical_PV_EE: 2035 - Année normale - Projection verte - Efficacité maximale
- 2035_typical_UB_E: 2035 - Année normale - Ultra bas carbone - Efficacité standard
- 2035_typical_UB_EE: 2035 - Année normale - Ultra bas carbone - Efficacité maximale
- 2035_cold_PV_E: 2035 - Année froide - Projection verte - Efficacité standard
- 2035_cold_PV_EE: 2035 - Année froide - Projection verte - Efficacité maximale
- 2035_cold_UB_E: 2035 - Année froide - Ultra bas carbone - Efficacité standard
- 2035_cold_UB_EE: 2035 - Année froide - Ultra bas carbone - Efficacité maximale


In [3]:
# Fonction pour exécuter une simulation pour un scénario donné
def run_simulation(scenario_key, verbose=True):
    """Exécute une simulation pour un scénario futur donné.
    
    Args:
        scenario_key: Clé du scénario (ex: '2035_warm_PV_E')
        verbose: Si True, affiche la sortie en temps réel
        
    Returns:
        Tuple (success, output) indiquant si la simulation a réussi et la sortie
    """
    # Vérifier que le scénario est valide
    if scenario_key not in config.future_scenarios.SCENARIOS:
        print(f"Erreur: Scénario {scenario_key} non valide")
        return False, f"Scénario {scenario_key} non valide"
    
    # Préparer la commande
    cmd = [sys.executable, "-m", "src.main", "simulate", "--future-scenario", scenario_key]
    start_time = time.time()
    
    # Exécuter la commande
    print(f"Démarrage de la simulation pour {scenario_key}...")
    process = subprocess.Popen(
        cmd, 
        stdout=subprocess.PIPE, 
        stderr=subprocess.STDOUT,
        cwd=project_root,
        universal_newlines=True
    )
    
    # Capture de la sortie
    output_lines = []
    for line in process.stdout:
        output_lines.append(line.rstrip())
        if verbose:
            print(line.rstrip())
    
    # Attendre la fin du processus
    process.wait()
    end_time = time.time()
    duration = end_time - start_time
    
    # Vérifier le résultat
    success = process.returncode == 0
    status = "Réussie" if success else "Échouée"
    
    print(f"Simulation {status} pour {scenario_key} en {duration:.1f} secondes")
    return success, "\n".join(output_lines)

In [4]:
# Fonction pour exécuter plusieurs simulations en séquence
def run_multiple_simulations(scenarios, verbose=False):
    """Exécute plusieurs simulations en séquence.
    
    Args:
        scenarios: Liste des scénarios à simuler
        verbose: Si True, affiche la sortie en temps réel
        
    Returns:
        Dictionnaire des résultats {scenario: (success, output)}
    """
    results = {}
    start_time = time.time()
    
    for i, scenario in enumerate(scenarios, 1):
        print(f"\n[{i}/{len(scenarios)}] Simulation du scénario {scenario}")
        success, output = run_simulation(scenario, verbose=verbose)
        results[scenario] = (success, output)
        
        # Afficher un résumé après chaque simulation
        if not verbose:
            status = "✅ Réussie" if success else "❌ Échouée"
            print(f"{status}: {scenario}")
    
    # Afficher un résumé final
    end_time = time.time()
    total_duration = end_time - start_time
    success_count = sum(1 for success, _ in results.values() if success)
    
    print(f"\nRésumé des simulations:")
    print(f"- Nombre total de simulations: {len(scenarios)}")
    print(f"- Simulations réussies: {success_count}")
    print(f"- Simulations échouées: {len(scenarios) - success_count}")
    print(f"- Durée totale: {total_duration:.1f} secondes ({total_duration/60:.1f} minutes)")
    
    return results

## 1. Simulation individuelle

Cette section permet de simuler un scénario futur spécifique.

In [5]:
# Sélectionner un scénario et l'exécuter
scenario_key = "2050_cold_UB_E"  # À modifier selon le scénario souhaité
success, output = run_simulation(scenario_key, verbose=True)

Démarrage de la simulation pour 2050_cold_UB_E...
2025-04-07 17:26:50,291 - src.calibration.transfer_learning_manager - INFO - Base de connaissances chargÃ©e avec 3 entrÃ©es
2025-04-07 17:26:50,291 - src.calibration.transfer_learning_manager - INFO - Chargement du modÃ¨le de transfert existant
2025-04-07 17:26:50,298 - src.calibration.transfer_learning_manager - INFO - ModÃ¨le de transfert chargÃ© avec succÃ¨s
2025-04-07 17:26:50315 - src.scenario_manager - INFO - InitialisÃ© 24 scÃ©narios futurs
2025-04-07 17:26:50318 - src.scenario_manager - INFO - Mapping mÃ©tÃ©o futur chargÃ©: 5 zones
2025-04-07 17:26:50318 - src.managers.simulation_manager - INFO - Tentative de chargement des paramÃ¨tres de calibration pour l'annÃ©e 2023
2025-04-07 17:26:50318 - src.managers.simulation_manager - INFO - RÃ©pertoire de calibration: c:\PI4\6\residence_quebec\data\outputs\calibration\2023
2025-04-07 17:26:50703 - src.managers.simulation_manager - INFO - Fichier de rÃ©sultats de calibration le plus rÃ©

## 2. Simulation de tous les scénarios

Cette section permet de simuler tous les scénarios futurs en séquence.

In [6]:
# Définir les scénarios à exécuter (tous par défaut)
# Vous pouvez filtrer cette liste pour ne sélectionner que certains scénarios
scenarios_to_run = config.future_scenarios.SCENARIOS.copy()

# Exemple de filtre pour simuler uniquement les scénarios 2035
# scenarios_to_run = [s for s in scenarios_to_run if s.startswith('2035')]

print(f"Scénarios à simuler ({len(scenarios_to_run)}):\n- " + "\n- ".join(scenarios_to_run))

Scénarios à simuler (24):
- 2035_warm_PV_E
- 2035_warm_PV_EE
- 2035_warm_UB_E
- 2035_warm_UB_EE
- 2035_typical_PV_E
- 2035_typical_PV_EE
- 2035_typical_UB_E
- 2035_typical_UB_EE
- 2035_cold_PV_E
- 2035_cold_PV_EE
- 2035_cold_UB_E
- 2035_cold_UB_EE
- 2050_warm_PV_E
- 2050_warm_PV_EE
- 2050_warm_UB_E
- 2050_warm_UB_EE
- 2050_typical_PV_E
- 2050_typical_PV_EE
- 2050_typical_UB_E
- 2050_typical_UB_EE
- 2050_cold_PV_E
- 2050_cold_PV_EE
- 2050_cold_UB_E
- 2050_cold_UB_EE


In [7]:
# Exécuter tous les scénarios sélectionnés
# Note: Cela peut prendre beaucoup de temps!
# Mettre verbose=False pour n'afficher que les résumés
all_results = run_multiple_simulations(scenarios_to_run, verbose=True)


[1/24] Simulation du scénario 2035_warm_PV_E
Démarrage de la simulation pour 2035_warm_PV_E...
2025-04-07 17:31:30,204 - src.calibration.transfer_learning_manager - INFO - Base de connaissances chargÃ©e avec 3 entrÃ©es
2025-04-07 17:31:30,204 - src.calibration.transfer_learning_manager - INFO - Chargement du modÃ¨le de transfert existant
2025-04-07 17:31:30,209 - src.calibration.transfer_learning_manager - INFO - ModÃ¨le de transfert chargÃ© avec succÃ¨s
2025-04-07 17:31:30236 - src.scenario_manager - INFO - InitialisÃ© 24 scÃ©narios futurs
2025-04-07 17:31:30237 - src.scenario_manager - INFO - Mapping mÃ©tÃ©o futur chargÃ©: 5 zones
2025-04-07 17:31:30237 - src.managers.simulation_manager - INFO - Tentative de chargement des paramÃ¨tres de calibration pour l'annÃ©e 2023
2025-04-07 17:31:30237 - src.managers.simulation_manager - INFO - RÃ©pertoire de calibration: c:\PI4\6\residence_quebec\data\outputs\calibration\2023
2025-04-07 17:31:30598 - src.managers.simulation_manager - INFO - Fi

KeyboardInterrupt: 

## 3. Analyse des résultats

Cette section permet d'explorer et visualiser les résultats des simulations.

In [8]:
# Fonction pour charger les résultats d'un scénario
def load_scenario_results(scenario_key):
    """Charge les résultats d'un scénario.
    
    Args:
        scenario_key: Clé du scénario (ex: '2035_warm_PV_E')
        
    Returns:
        Tuple (provincial_results, provincial_summary, zones)
    """
    # Extraire l'année du scénario
    year = config.future_scenarios.get_scenario_year(scenario_key)
    
    # Construire les chemins
    base_dir = config.paths['output'].get_simulation_dir(year, scenario_key)
    provincial_dir = base_dir / 'provincial'
    zones_dir = base_dir / 'zones'
    
    # Vérifier que les fichiers existent
    if not provincial_dir.exists() or not (provincial_dir / 'hourly.csv').exists():
        print(f"Erreur: Résultats provinciaux non trouvés pour {scenario_key}")
        return None, None, {}
    
    # Charger les résultats provinciaux
    provincial_results = pd.read_csv(provincial_dir / 'hourly.csv')
    provincial_results['Time'] = pd.to_datetime(provincial_results['Time'])
    provincial_results.set_index('Time', inplace=True)
    
    # Charger le résumé provincial
    with open(provincial_dir / 'summary.json', 'r') as f:
        provincial_summary = json.load(f)
    
    # Charger les résultats des zones
    zones = {}
    if zones_dir.exists():
        for zone_dir in zones_dir.glob('zone_*'):
            zone_id = int(zone_dir.name.replace('zone_', ''))
            if (zone_dir / 'hourly.csv').exists():
                zone_results = pd.read_csv(zone_dir / 'hourly.csv')
                zone_results['Time'] = pd.to_datetime(zone_results['Time'])
                zone_results.set_index('Time', inplace=True)
                
                # Charger le résumé de la zone
                try:
                    with open(zone_dir / 'summary.json', 'r') as f:
                        zone_summary = json.load(f)
                except:
                    zone_summary = {}
                    
                zones[zone_id] = {
                    'results': zone_results,
                    'summary': zone_summary
                }
    
    return provincial_results, provincial_summary, zones

In [ ]:
# Sélectionner un scénario pour l'analyse
scenario_to_analyze = "2050_warm_UB_E" 

# Charger les résultats
results, summary, zones = load_scenario_results(scenario_to_analyze)

if results is not None:
    # Afficher le résumé
    print(f"Résumé du scénario {scenario_to_analyze}:")
    annual_consumption = summary.get('annual_consumption', {})
    total_kwh = annual_consumption.get('total_kwh', 0)
    total_gwh = annual_consumption.get('total_gwh', 0)
    print(f"- Consommation totale: {total_gwh:.1f} GWh ({total_kwh:.0f} kWh)")
    
    # Afficher les usages finaux
    print("\nRépartition par usage final:")
    end_uses = {k: v for k, v in annual_consumption.items() if '_kwh' in k and k != 'total_kwh'}
    for use, kwh in end_uses.items():
        use_name = use.replace('_kwh', '').replace('_', ' ').title()
        percentage = (kwh / total_kwh * 100) if total_kwh > 0 else 0
        print(f"- {use_name}: {kwh/1e6:.1f} GWh ({percentage:.1f}%)")
    
    # Afficher les zones
    print(f"\nZones: {len(zones)}")
    
    # Générer un graphique pour la consommation journalière
    plt.figure(figsize=(12, 6))
    daily = results['Fuel Use: Electricity: Total'].resample('D').sum()
    plt.plot(daily.index, daily.values, label='Consommation journalière')
    plt.title(f"Consommation journalière - {scenario_to_analyze}")
    plt.xlabel("Date")
    plt.ylabel("Électricité (kWh)")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Générer un graphique pour les usages finaux mensuels
    end_use_columns = [col for col in results.columns if col.startswith('End Use: Electricity:')]
    if end_use_columns:
        monthly_end_uses = results[end_use_columns].resample('M').sum()
        monthly_end_uses.columns = [col.replace('End Use: Electricity: ', '') for col in monthly_end_uses.columns]
        
        plt.figure(figsize=(14, 7))
        monthly_end_uses.plot(kind='bar', stacked=True, ax=plt.gca())
        plt.title(f"Consommation mensuelle par usage - {scenario_to_analyze}")
        plt.xlabel("Mois")
        plt.ylabel("Électricité (kWh)")
        plt.legend(title="Usages finaux")
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        plt.show()

## 4. Comparaison entre scénarios

Cette section permet de comparer plusieurs scénarios entre eux.

In [ ]:
# Sélectionner les scénarios à comparer
scenarios_to_compare = [
    "2035_warm_PV_E",
    "2035_warm_PV_EE",
    "2050_warm_PV_E",
    "2050_warm_PV_EE"
]  # À modifier selon les scénarios à comparer

# Charger les résultats de tous les scénarios
scenario_data = {}
for scenario in scenarios_to_compare:
    results, summary, _ = load_scenario_results(scenario)
    if results is not None and summary is not None:
        scenario_data[scenario] = {
            'results': results,
            'summary': summary
        }

if scenario_data:
    # Comparer la consommation totale
    consumption_comparison = [
        {
            'Scénario': scenario,
            'Consommation (GWh)': data['summary'].get('annual_consumption', {}).get('total_gwh', 0),
            'Consommation (TWh)': data['summary'].get('annual_consumption', {}).get('total_gwh', 0) / 1000,
            'Description': config.future_scenarios.SCENARIO_NAMES.get(scenario, scenario)
        }
        for scenario, data in scenario_data.items()
    ]
    
    # Créer un DataFrame pour la comparaison
    comparison_df = pd.DataFrame(consumption_comparison)
    display(comparison_df)
    
    # Visualiser la comparaison
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Scénario', y='Consommation (GWh)', data=comparison_df)
    plt.title("Comparaison de la consommation électrique annuelle")
    plt.ylabel("Consommation (GWh)")
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
    
    # Comparer les profils mensuels
    plt.figure(figsize=(14, 7))
    for scenario, data in scenario_data.items():
        monthly = data['results']['Fuel Use: Electricity: Total'].resample('M').sum() / 1e6  # Conversion en GWh
        plt.plot(monthly.index, monthly.values, label=scenario)
    
    plt.title("Comparaison des profils mensuels")
    plt.xlabel("Mois")
    plt.ylabel("Consommation (GWh)")
    plt.legend(title="Scénarios")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 5. Impact du changement climatique (2035 vs 2050)

Cette section permet d'analyser l'impact du changement climatique en comparant des scénarios avec les mêmes paramètres mais à différents horizons temporels.

In [ ]:
# Comparer les scénarios 2035 vs 2050 avec mêmes paramètres
climate_comparison_sets = [
    # Format: (scénario 2035, scénario 2050, label)
    ("2035_warm_PV_E", "2050_warm_PV_E", "Warm climate, PV, E"),
    ("2035_cold_PV_E", "2050_cold_PV_E", "Cold climate, PV, E"),
    ("2035_typical_PV_E", "2050_typical_PV_E", "Typical climate, PV, E")
]

for s2035, s2050, label in climate_comparison_sets:
    # Charger les résultats
    data_2035 = load_scenario_results(s2035)
    data_2050 = load_scenario_results(s2050)
    
    if data_2035[0] is not None and data_2050[0] is not None:
        # Comparer la consommation mensuelle
        monthly_2035 = data_2035[0]['Fuel Use: Electricity: Total'].resample('M').sum() / 1e6
        monthly_2050 = data_2050[0]['Fuel Use: Electricity: Total'].resample('M').sum() / 1e6
        
        # Ajuster pour avoir le même index
        monthly_2035.index = [idx.replace(year=2023) for idx in monthly_2035.index]
        monthly_2050.index = [idx.replace(year=2023) for idx in monthly_2050.index]
        
        # Visualiser
        plt.figure(figsize=(14, 7))
        plt.plot(monthly_2035.index, monthly_2035.values, label='2035', color='blue')
        plt.plot(monthly_2050.index, monthly_2050.values, label='2050', color='red')
        plt.title(f"Impact du changement climatique: {label}")
        plt.xlabel("Mois")
        plt.ylabel("Consommation (GWh)")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Calculer les différences
        diff = monthly_2050 - monthly_2035
        diff_pct = (monthly_2050 - monthly_2035) / monthly_2035 * 100
        
        # Afficher les différences
        plt.figure(figsize=(14, 7))
        plt.bar(diff.index, diff.values, color=['red' if x > 0 else 'blue' for x in diff.values])
        plt.title(f"Différence 2050-2035: {label}")
        plt.xlabel("Mois")
        plt.ylabel("Différence (GWh)")
        plt.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        plt.show()
        
        # Afficher un résumé
        total_2035 = data_2035[1]['annual_consumption']['total_gwh']
        total_2050 = data_2050[1]['annual_consumption']['total_gwh']
        diff_total = total_2050 - total_2035
        diff_pct_total = (total_2050 - total_2035) / total_2035 * 100
        
        print(f"\nRésumé pour {label}:")
        print(f"- Total 2035: {total_2035:.1f} GWh")
        print(f"- Total 2050: {total_2050:.1f} GWh")
        print(f"- Différence: {diff_total:.1f} GWh ({diff_pct_total:.1f}%)")

## 6. Impact de l'efficacité énergétique (E vs EE)

Cette section permet d'analyser l'impact des mesures d'efficacité énergétique en comparant des scénarios avec efficacité standard (E) et maximale (EE).

In [ ]:
# Comparer les scénarios E vs EE
efficiency_comparison_sets = [
    # Format: (scénario E, scénario EE, label)
    ("2035_warm_PV_E", "2035_warm_PV_EE", "2035, Warm climate, PV"),
    ("2050_warm_PV_E", "2050_warm_PV_EE", "2050, Warm climate, PV"),
    ("2035_typical_UB_E", "2035_typical_UB_EE", "2035, Typical climate, UB")
]

# DataFrame pour stocker les résultats de l'analyse d'efficacité
efficiency_results = []

for s_std, s_eff, label in efficiency_comparison_sets:
    # Charger les résultats
    data_std = load_scenario_results(s_std)
    data_eff = load_scenario_results(s_eff)
    
    if data_std[0] is not None and data_eff[0] is not None:
        # Comparer la consommation mensuelle
        monthly_std = data_std[0]['Fuel Use: Electricity: Total'].resample('M').sum() / 1e6
        monthly_eff = data_eff[0]['Fuel Use: Electricity: Total'].resample('M').sum() / 1e6
        
        # Visualiser
        plt.figure(figsize=(14, 7))
        plt.plot(monthly_std.index, monthly_std.values, label='Efficacité standard (E)', color='orange')
        plt.plot(monthly_eff.index, monthly_eff.values, label='Efficacité maximale (EE)', color='green')
        plt.title(f"Impact de l'efficacité énergétique: {label}")
        plt.xlabel("Mois")
        plt.ylabel("Consommation (GWh)")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Calculer les économies d'énergie
        savings = monthly_std - monthly_eff
        savings_pct = (monthly_std - monthly_eff) / monthly_std * 100
        
        # Visualiser les économies
        plt.figure(figsize=(14, 7))
        plt.bar(savings.index, savings.values, color='green')
        plt.title(f"Économies d'énergie (E-EE): {label}")
        plt.xlabel("Mois")
        plt.ylabel("Économies (GWh)")
        plt.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        plt.show()
        
        # Afficher un résumé
        total_std = data_std[1]['annual_consumption']['total_gwh']
        total_eff = data_eff[1]['annual_consumption']['total_gwh']
        savings_total = total_std - total_eff
        savings_pct_total = (total_std - total_eff) / total_std * 100
        
        print(f"\nRésumé pour {label}:")
        print(f"- Total avec efficacité standard (E): {total_std:.1f} GWh")
        print(f"- Total avec efficacité maximale (EE): {total_eff:.1f} GWh")
        print(f"- Économies: {savings_total:.1f} GWh ({savings_pct_total:.1f}%)")
        
        # Ajouter au DataFrame des résultats
        efficiency_results.append({
            'Scénario': label,
            'Standard (GWh)': total_std,
            'Efficace (GWh)': total_eff,
            'Économies (GWh)': savings_total,
            'Économies (%)': savings_pct_total
        })

# Afficher le tableau récapitulatif
if efficiency_results:
    efficiency_df = pd.DataFrame(efficiency_results)
    display(efficiency_df)

## 7. Exportation des résultats

Cette section permet d'exporter les résultats des analyses pour une utilisation ultérieure.

In [ ]:
# Créer un rapport récapitulatif des scénarios simulés
def create_scenarios_summary(scenarios=None):
    """Crée un résumé de tous les scénarios simulés.
    
    Args:
        scenarios: Liste des scénarios à inclure (tous par défaut)
        
    Returns:
        DataFrame avec le résumé des scénarios
    """
    if scenarios is None:
        scenarios = config.future_scenarios.SCENARIOS
    
    summary_data = []
    
    for scenario in scenarios:
        # Extraire l'année du scénario
        year = config.future_scenarios.get_scenario_year(scenario)
        
        # Vérifier si le scénario a été simulé
        results_dir = config.paths['output'].get_simulation_dir(year, scenario)
        provincial_dir = results_dir / 'provincial'
        
        if provincial_dir.exists() and (provincial_dir / 'summary.json').exists():
            # Charger le résumé provincial
            with open(provincial_dir / 'summary.json', 'r') as f:
                summary = json.load(f)
                
            # Extraire les informations importantes
            total_gwh = summary.get('annual_consumption', {}).get('total_gwh', 0)
            
            # Extraire les usages finaux
            end_uses = {}
            for key, value in summary.get('annual_consumption', {}).items():
                if '_kwh' in key and key != 'total_kwh':
                    end_use = key.replace('_kwh', '')
                    end_uses[end_use] = value / 1e6  # Conversion en GWh
            
            # Décomposer le scénario
            parts = scenario.split('_')
            climate_type = parts[1] if len(parts) > 1 else ""
            growth_type = parts[2] if len(parts) > 2 else ""
            efficiency_type = parts[3] if len(parts) > 3 else ""
            
            # Ajouter au résumé
            scenario_summary = {
                'Scénario': scenario,
                'Année': year,
                'Climat': climate_type,
                'Croissance': growth_type,
                'Efficacité': efficiency_type,
                'Consommation (GWh)': total_gwh
            }
            
            # Ajouter les usages finaux
            scenario_summary.update({f"{use} (GWh)": value for use, value in end_uses.items()})
            
            summary_data.append(scenario_summary)
    
    # Créer le DataFrame
    if summary_data:
        return pd.DataFrame(summary_data)
    else:
        return pd.DataFrame()

# Générer et afficher le résumé
scenarios_summary = create_scenarios_summary()
if not scenarios_summary.empty:
    # Trier par année et type de climat
    scenarios_summary = scenarios_summary.sort_values(['Année', 'Climat', 'Croissance', 'Efficacité'])
    display(scenarios_summary)
    
    # Sauvegarder en CSV
    export_path = project_root / 'notebooks' / 'scenario_results_summary.csv'
    scenarios_summary.to_csv(export_path, index=False)
    print(f"Résumé exporté vers: {export_path}")
else:
    print("Aucun scénario simulé trouvé dans les résultats.")

In [ ]:
import os
import shutil
from pathlib import Path

def clean_directories(base_path):
    """
    Parcourt la structure hiérarchique et supprime tous les dossiers
    qui ne s'appellent pas 'mrc' au niveau des sous-dossiers B2.
    
    Args:
        base_path (str): Chemin du dossier de base B
    """
    base_dir = Path(base_path)
    
    # Vérifier que le dossier de base existe
    if not base_dir.is_dir():
        print(f"Le dossier {base_dir} n'existe pas.")
        return
    
    # Parcourir les dossiers de niveau B1
    for b1_dir in base_dir.iterdir():
        if not b1_dir.is_dir():
            continue
            
        print(f"Traitement du dossier B1: {b1_dir.name}")
        
        # Parcourir les dossiers de niveau B2
        for b2_dir in b1_dir.iterdir():
            if not b2_dir.is_dir():
                continue
                
            print(f"  Traitement du dossier B2: {b2_dir.name}")
            
            # Parcourir et supprimer les sous-dossiers qui ne s'appellent pas 'mrc'
            for sub_dir in b2_dir.iterdir():
                if sub_dir.is_dir() and sub_dir.name != "mrc":
                    print(f"    Suppression de: {sub_dir.name}")
                    shutil.rmtree(sub_dir)

if __name__ == "__main__":
    # Remplacez ce chemin par votre dossier B
    base_path = Path(r"C:\PI4\6\residence_quebec\data\outputs\simulations")
    
    # Confirmation avant de commencer
    print(f"Ce script va supprimer tous les dossiers qui ne s'appellent pas 'mrc' "
          f"dans les sous-dossiers de {base_path}.")
    confirmation = "oui"
    
    if confirmation.lower() in ["oui", "o", "yes", "y"]:
        clean_directories(base_path)
        print("Nettoyage terminé.")
    else:
        print("Opération annulée.")